In [1]:
!pip install torch==1.12.1
!pip install fastai==2.7.9
!pip install pytorch_benchmark==0.3.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cpu
    Uninstalling torch-2.0.0+cpu:
      Successfully uninstalled torch-2.0.0+cpu
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cpu requires torch==2.0.0, but you have torch 1.12.1 which is incompatible.
torchtext 0.15.1+cpu requires torch==2.0.0, but you have torch 1.12.1 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.12.1 which is incompatible.
torchaudio 2.0.1+cpu requires torch==2.0.0, but you have torch 1.12.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 75.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from pytorch_benchmark import benchmark
from fastai.vision.all import *
import gc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [38]:
# Create your learner and load your model       
model = load_learner("/kaggle/input/testingpklforinference/VGG16_bn_native_fp16_accum2_sparse25_pruned.pkl")

# Set the model to evaluation mode
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(96, 192, kern

In [39]:
# Make predictions on multiple batches of data and measure the time
sample = torch.randn(1, 3, 64, 64)  # (B, C, H, W)
results = benchmark(model.model, sample, num_runs=1000)

Warming up with batch_size=1: 100%|██████████| 1/1 [00:00<00:00, 11.93it/s]


Warning! No positional inputs found for a module, assuming batch size is 1.


Measuring inference for batch_size=1: 100%|██████████| 1000/1000 [00:37<00:00, 26.34it/s]


In [40]:
results

{'machine_info': {'system': {'system': 'Linux',
   'node': '7f6ba1670c29',
   'release': '5.15.90+'},
  'cpu': {'model': 'Intel(R) Xeon(R) CPU @ 2.20GHz',
   'architecture': 'x86_64',
   'cores': {'physical': 2, 'total': 4},
   'frequency': '0.00 GHz'},
  'memory': {'total': '31.36 GB', 'used': '6.64 GB', 'available': '24.27 GB'},
  'gpus': None},
 'device': 'cpu',
 'params': 102181178,
 'flops': 803618314,
 'timing': {'batch_size_1': {'on_device_inference': {'metrics': {'batches_per_second_mean': 26.779806537120162,
     'batches_per_second_std': 1.4370222519250007,
     'batches_per_second_min': 14.02842942478912,
     'batches_per_second_max': 27.95773981322864,
     'seconds_per_batch_mean': 0.03749026322364807,
     'seconds_per_batch_std': 0.0028093145869135003,
     'seconds_per_batch_min': 0.03576827049255371,
     'seconds_per_batch_max': 0.07128381729125977},
    'human_readable': {'batches_per_second': '26.78 +/- 1.44 [14.03, 27.96]',
     'batch_latency': '37.490 ms +/- 2.8

In [41]:
# Clear the learner model and trigger garbage collection
del model
del sample
del results
gc.collect()

395